In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
import pandas as pd
import pickle as pkl

In [3]:
data = pd.read_csv(r'../datasets/RF-dataset.csv')
for colname in ['name', 'keyword', 'common']:
    data[colname] = data[colname].astype(int)
data['label'] = [1 if x == 'nominal' else 0 for x in data['label']]
#X, y = data.iloc[:, :-1], data['label']
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_train, X_test, y_train, y_test = data.iloc[:101, :-1], data.iloc[102:, :-1], data.iloc[:101, -1], data.iloc[102:, -1] 
print(X_train)
print(y_train)

      size     ratio  name  keyword  common
0      480  0.020833     0        0       0
1      480  0.022917     1        0       0
2      480  0.006250     0        1       0
3      480  0.010417     0        0       0
4      480  0.006250     0        0       0
..     ...       ...   ...      ...     ...
96   48895  0.000102     0        0       0
97   48895  0.004520     0        0       0
98   48895  0.000061     1        0       0
99    1000  0.002000     1        0       0
100  18207  0.005492     0        0       0

[101 rows x 5 columns]
0      1
1      1
2      0
3      0
4      1
      ..
96     1
97     1
98     1
99     1
100    0
Name: label, Length: 101, dtype: int64


In [4]:
#clf = RandomForestClassifier(n_estimators=300)
clf = GradientBoostingClassifier(max_depth=2)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
print("F1:", metrics.f1_score(y_test, y_pred))

Accuracy: 0.9523809523809523
F1: 0.967741935483871


In [5]:
pkl.dump(clf, open('trained_gbc.sav', 'wb'))